# MODULO_04 AJUSTE EMBEDDINGS CON TERMINOS TECNICOS

# ---------------------------------------------
# ATENCIÓN - FIJAR ESTAS VARIABLES ANTES DE EJECUTAR
# ---------------------------------------------

In [1]:

nombre_lote = "LOTE_20250614"

nombre_modulo = "MODULO_04"

In [2]:
# ---------------------------------------------
# Configuración del entorno (Colab y Local)
# ---------------------------------------------

try:
    import google.colab
    EN_COLAB = True
except ImportError:
    EN_COLAB = False

if EN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    ruta_base = "/content/drive/MyDrive/TFM_EVA_MARTIN/Modulos"
else:
    ruta_base = "G:/Mi unidad/TFM_EVA_MARTIN/Modulos"

print(f"Entorno detectado: {'Google Colab' if EN_COLAB else 'Local'}")
print(f"Ruta base: {ruta_base}")

lote_id = nombre_lote.replace("LOTE_", "")

Mounted at /content/drive
Entorno detectado: Google Colab
Ruta base: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos


In [3]:
import sys
import os
ruta_config = os.path.join(ruta_base, "config.yaml")

if ruta_base not in sys.path:
    sys.path.append(ruta_base)
import yaml

# Cargar configuración desde el archivo YAML
with open(ruta_config, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

config = yaml.safe_load(open(ruta_config))

# Extraer bloque de parámetros (KeyError si falta alguna clave)
params = config["parametros"]



# Carga utilidades comunes e inicialización del entorno

In [4]:
import pandas as pd

import utilidades_comunes

# 1. Configurar logger
import logging
logger = utilidades_comunes.configurar_logger(nombre_modulo, ruta_logs=os.path.join(ruta_base, nombre_modulo, "logs"))
logger.setLevel(logging.DEBUG)

# 2. Inicializar entorno
entorno = utilidades_comunes.inicializar_entorno(nombre_modulo, nombre_lote, ruta_base, ruta_config, logger=logger)


2025-06-28 13:50:56,323 - INFO - 📁 Entorno inicializado para MODULO_04
INFO:MODULO_04:📁 Entorno inicializado para MODULO_04
2025-06-28 13:50:56,325 - INFO - 📂 Ruta entrada: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_03/./salida
INFO:MODULO_04:📂 Ruta entrada: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_03/./salida
2025-06-28 13:50:56,327 - INFO - 📂 Ruta salida: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./salida
INFO:MODULO_04:📂 Ruta salida: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./salida
2025-06-28 13:50:56,328 - INFO - 📂 Ruta logs: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./logs
INFO:MODULO_04:📂 Ruta logs: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./logs
2025-06-28 13:50:56,329 - INFO - 📂 Ruta ejemplos: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./ejemplos
INFO:MODULO_04:📂 Ruta ejemplos: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./ejemplos
2025-06-28 13:50:56,330 - INFO - 🔗 Módulo 

In [5]:
# ---------------------------------------------
# 2. Cargar términos técnicos desde JSON
# ---------------------------------------------
import json
ruta_json = os.path.join(entorno["ruta_entrada"], "terminos_tecnicos.json")
with open(ruta_json, "r", encoding="utf-8") as f:
    terminos_tecnicos = json.load(f)

logger.info(f"📄 Términos técnicos cargados: {len(terminos_tecnicos)}")

2025-06-28 13:51:00,607 - INFO - 📄 Términos técnicos cargados: 14
INFO:MODULO_04:📄 Términos técnicos cargados: 14


# Carga y analisis dataset de entrada

In [7]:
patron_busqueda = os.path.join(
    entorno["ruta_entrada"],
    f"dataset_{entorno['nombre_modulo_anterior'].lower()}_{entorno['lote_id']}*.csv"
)

import glob
archivos_encontrados = glob.glob(patron_busqueda)

if not archivos_encontrados:
    raise FileNotFoundError(f"No se encontró archivo de entrada para el lote {nombre_lote} con patrón: {patron_busqueda}")

fichero_entrada = archivos_encontrados[0]
df_entrada = utilidades_comunes.cargar_dataset(fichero_entrada, logger=logger)

utilidades_comunes.mostrar_muestra_dataset(df_entrada, "dataset de entrada", logger=logger)
utilidades_comunes.guardar_muestra_dataset(df_entrada, "entrada", entorno["ruta_ejemplos"], logger=logger, n=1)


df = df_entrada.copy()

2025-06-28 13:51:29,343 - INFO - ✅ Dataset cargado desde /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_03/./salida/dataset_modulo_03_20250614.csv (140 filas, 4 columnas)
INFO:MODULO_04:✅ Dataset cargado desde /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_03/./salida/dataset_modulo_03_20250614.csv (140 filas, 4 columnas)
2025-06-28 13:51:29,344 - INFO - --- Muestra de dataset de entrada (primeras 5 filas) ---
INFO:MODULO_04:--- Muestra de dataset de entrada (primeras 5 filas) ---
2025-06-28 13:51:29,346 - INFO - Filas totales: 140, Columnas totales: 4
INFO:MODULO_04:Filas totales: 140, Columnas totales: 4
2025-06-28 13:51:29,369 - INFO - 
| nomfichero                                                                                            | etiqueta   | transcripcion                                                                                                                                                                                                                

In [12]:
# ---------------------------------------------
# 3. Preparación de textos para ajuste de embeddings (sin división si es pequeño)
# ---------------------------------------------
from collections import Counter

# Limpieza y mapeo de etiquetas
etiquetas_map = {"Positivo": 0, "Neutro": 1, "Negativo": 2}
df = df.dropna(subset=["texto_etiquetado"])
df["texto_etiquetado"] = df["texto_etiquetado"].astype(str).str.strip()
df["label"] = df["etiqueta"].map(etiquetas_map)

# Comprobamos si se puede dividir de forma segura
conteo = Counter(df["label"])
minimo_por_clase = min(conteo.values())

if minimo_por_clase < 3 or len(df) < 30:
    logger.warning("⚠️ Dataset demasiado pequeño para dividir. Usando todo como train_texts.")
    train_texts = df["texto_etiquetado"].tolist()
else:
    from sklearn.model_selection import train_test_split

    df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
    df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp["label"])

    train_texts = df_train["texto_etiquetado"].tolist()
    val_texts   = df_val["texto_etiquetado"].tolist()
    test_texts  = df_test["texto_etiquetado"].tolist()

    train_labels = df_train["label"].tolist()
    val_labels   = df_val["label"].tolist()
    test_labels  = df_test["label"].tolist()

    # Guardar subconjuntos
    df_train.to_csv(os.path.join(entorno["ruta_salida"], "train.csv"), index=False)
    df_val.to_csv(os.path.join(entorno["ruta_salida"], "val.csv"), index=False)
    df_test.to_csv(os.path.join(entorno["ruta_salida"], "test.csv"), index=False)

    logger.info(f"🗂 Dataset dividido: {len(train_texts)} train, {len(val_texts)} val, {len(test_texts)} test")

# Mostrar conteo de clases
print("\n📊 Número de muestras por clase:")
print(df["etiqueta"].value_counts())

# Mostrar algunos ejemplos de transcripciones
print("\n📝 Ejemplo de transcripciones:")
for i, fila in df.sample(n=min(2, len(df))).iterrows():  # muestra 2 ejemplos como máximo
    print(f"\n📁 {fila['nomfichero']}  [{fila['etiqueta']}]")
    print(f"{fila['texto_etiquetado'][:300]}...")
# Verificar que las conversiones se hicieron correctamente
print(df_train["label"].unique())
print(df_train["label"].dtype)


2025-06-28 13:54:15,815 - INFO - 🗂 Dataset dividido: 112 train, 14 val, 14 test
INFO:MODULO_04:🗂 Dataset dividido: 112 train, 14 val, 14 test



📊 Número de muestras por clase:
etiqueta
Positivo    95
Neutro      25
Negativo    20
Name: count, dtype: int64

📝 Ejemplo de transcripciones:

📁 Positivo_SD23-03590_[965021512]_MERIDIANO_900_GEN._2023-03-01_12-55-57_679839976_procesado_limpio.txt  [Positivo]
Agente: Sí, dígame. Agente: Hola, buenos días. ¿Doña CLIENTE? Agente: Sí, soy yo. Dígame. Agente: Mire, mi nombre es AGENTE, le llamo del Departamento de Asistencia de Seguros Meridiano. Doña CLIENTE, primero de todo transmitirle mis más sinceras condolencias por el fallecimiento de su tía, doña DIF...

📁 Positivo_SD23-03630_[965021512]_MERIDIANO_900_GEN._2023-03-10_11-59-17_647288618_procesado_limpio.txt  [Positivo]
Agente: Bienvenido a Meridiano. Por su seguridad, esta llamada podrá ser grabada. Usted consiente en que los datos que facilite se incorporen en un fichero titularidad de Meridiano S. A. con la finalidad de gestionar la prestación del servicio. Si ya conoce nuestra política de protección de datos y ...
[2 0 1]
int64


# PASO 4: Procesamiento específico del módulo

In [9]:
# ---------------------------------------------
# Funcion para detectar términos nuevos - porque los terminos técnicos
# son acumulativos
# ---------------------------------------------
def detectar_terminos_nuevos(terminos_tecnicos, ruta_tokenizer_anterior, logger=None):
    """
    Compara el vocabulario actual con los términos técnicos nuevos
    y devuelve únicamente los que aún no existen.

    Args:
        terminos_tecnicos (list): Lista de términos del JSON.
        ruta_tokenizer_anterior (str): Carpeta con tokenizer previo.
        logger (logging.Logger, optional): Para registrar mensajes.

    Returns:
        list: Lista de términos nuevos que deben añadirse.
    """
    from transformers import LongformerTokenizerFast

    if os.path.exists(ruta_tokenizer_anterior):
        tokenizer_tmp = LongformerTokenizerFast.from_pretrained(ruta_tokenizer_anterior)
        tokens_existentes = set(tokenizer_tmp.get_vocab().keys())
        if logger:
            logger.info("📥 Tokenizador anterior cargado para comparación de vocabulario.")
    else:
        tokenizer_tmp = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")
        tokens_existentes = set(tokenizer_tmp.get_vocab().keys())
        if logger:
            logger.info("📥 Tokenizador base cargado para comparación de vocabulario.")

    terminos_nuevos = [t for t in terminos_tecnicos if t not in tokens_existentes]
    if logger:
        logger.info(f"🆕 Términos nuevos detectados y añadidos: {len(terminos_nuevos)}")

    return terminos_nuevos


In [10]:
from datetime import datetime

def registrar_terminos_en_log(terminos_nuevos, lote_id, ruta_salida, logger=None):
    if not terminos_nuevos:
        if logger:
            logger.info("📋 No hay términos nuevos que registrar en el log.")
        return

    fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    registros = [{"termino": t, "lote_id": lote_id, "fecha_hora": fecha} for t in terminos_nuevos]
    df_nuevos = pd.DataFrame(registros)

    ruta_log = os.path.join(ruta_salida, "registro_terminos_por_lote.csv")
    if os.path.exists(ruta_log):
        df_log = pd.read_csv(ruta_log)
        df_final = pd.concat([df_log, df_nuevos], ignore_index=True).drop_duplicates(subset=["termino"])
    else:
        df_final = df_nuevos

    df_final.to_csv(ruta_log, index=False, encoding="utf-8")
    if logger:
        logger.info(f"🗂 Registro actualizado: {len(terminos_nuevos)} términos añadidos al log.")


In [11]:
# ─── 4. Cargar modelo y tokenizador Longformer ───
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification
import torch

logger.info("🔧 Cargando modelo y tokenizador Longformer…")
tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)

# Añadir nuevos tokens y redimensionar embeddings
modelo_dir_prev = os.path.join(entorno["ruta_entrada"], "modelo_ajustado")
terminos_nuevos = detectar_terminos_nuevos(terminos_tecnicos, modelo_dir_prev, logger=logger)
tokenizer.add_tokens(terminos_nuevos)
model.resize_token_embeddings(len(tokenizer))

# Congelar todo excepto embeddings
for param in model.parameters():
    param.requires_grad = False
model.longformer.embeddings.word_embeddings.weight.requires_grad = True

2025-06-28 13:52:24,276 - INFO - 🔧 Cargando modelo y tokenizador Longformer…
INFO:MODULO_04:🔧 Cargando modelo y tokenizador Longformer…
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-06-28 13:52:34,425 - INFO - 📥 Tokenizador base cargado para comparación de vocabulario.
INFO:MODULO_04:📥 Tokenizador base cargado para comparación de vocabulario.
2025-06-28 13:52:34,427 - INFO - 🆕 Términos nuevos detectados y añadidos: 13
INFO:MODULO_04:🆕 Términos nuevos detectados y añadidos: 13


model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [13]:
registrar_terminos_en_log(terminos_nuevos, lote_id, entorno["ruta_salida"], logger=logger)

2025-06-28 13:54:53,997 - INFO - 🗂 Registro actualizado: 13 términos añadidos al log.
INFO:MODULO_04:🗂 Registro actualizado: 13 términos añadidos al log.


In [14]:
# ---------------------------------------------
# 5. Entrenamiento de Embeddings
# ---------------------------------------------
from torch.utils.data import Dataset, DataLoader
# Si no hiciste división y no tienes train_labels, generamos etiquetas vacías o ficitcias
if "train_labels" not in locals() or len(train_labels) != len(train_texts):
    logger.warning("⚠️ No se encontraron etiquetas. Se asignarán etiquetas ficticias (0) a todo el dataset.")
    train_labels = [0] * len(train_texts)

# Tokenizar con truncado y padding explícito
inputs = tokenizer(
    train_texts,
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
)
# Añadir etiquetas
inputs["labels"] = torch.tensor(train_labels)

class SimpleDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

# Crear dataset y dataloader
dataset = SimpleDataset(inputs)
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Optimizador solo para los embeddings entrenables
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

logger.info("🚀 Iniciando ajuste de embeddings…")
model.train()
for epoch in range(2):
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        logger.info(f"Epoch {epoch} - Loss: {loss.item():.4f}")

2025-06-28 13:54:56,526 - INFO - 🚀 Iniciando ajuste de embeddings…
INFO:MODULO_04:🚀 Iniciando ajuste de embeddings…
Initializing global attention on CLS token...
2025-06-28 13:54:58,420 - INFO - Epoch 0 - Loss: 0.9700
INFO:MODULO_04:Epoch 0 - Loss: 0.9700
2025-06-28 13:54:59,915 - INFO - Epoch 0 - Loss: 1.2311
INFO:MODULO_04:Epoch 0 - Loss: 1.2311
2025-06-28 13:55:01,322 - INFO - Epoch 0 - Loss: 1.0805
INFO:MODULO_04:Epoch 0 - Loss: 1.0805
2025-06-28 13:55:02,879 - INFO - Epoch 0 - Loss: 1.1305
INFO:MODULO_04:Epoch 0 - Loss: 1.1305
2025-06-28 13:55:04,824 - INFO - Epoch 0 - Loss: 1.0169
INFO:MODULO_04:Epoch 0 - Loss: 1.0169
2025-06-28 13:55:06,262 - INFO - Epoch 0 - Loss: 1.0673
INFO:MODULO_04:Epoch 0 - Loss: 1.0673
2025-06-28 13:55:07,698 - INFO - Epoch 0 - Loss: 1.1415
INFO:MODULO_04:Epoch 0 - Loss: 1.1415
2025-06-28 13:55:09,138 - INFO - Epoch 0 - Loss: 1.0876
INFO:MODULO_04:Epoch 0 - Loss: 1.0876
2025-06-28 13:55:10,492 - INFO - Epoch 0 - Loss: 1.2066
INFO:MODULO_04:Epoch 0 - Loss:

In [15]:
# ---------------------------------------------
# 6. Guardar modelo ajustado
# ---------------------------------------------
modelo_dir = os.path.join(entorno["ruta_salida"], "modelo_ajustado")
os.makedirs(modelo_dir, exist_ok=True)
model.save_pretrained(modelo_dir)
tokenizer.save_pretrained(modelo_dir)
logger.info(f"💾 Modelo y tokenizador ajustados guardados en: {modelo_dir}")

logger.info(f"✅ Finalización del procesamiento del módulo {nombre_modulo}")

2025-06-28 14:00:37,345 - INFO - 💾 Modelo y tokenizador ajustados guardados en: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./salida/modelo_ajustado
INFO:MODULO_04:💾 Modelo y tokenizador ajustados guardados en: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./salida/modelo_ajustado
2025-06-28 14:00:37,346 - INFO - ✅ Finalización del procesamiento del módulo MODULO_04
INFO:MODULO_04:✅ Finalización del procesamiento del módulo MODULO_04


In [16]:
print("🧾 Tokens añadidos al vocabulario:")
print(terminos_tecnicos[:10])  # primeros 10

print(f"\n🔢 Tamaño total del vocabulario tras expansión: {len(tokenizer)}")

# Mostrar ejemplo de cómo el modelo tokeniza una frase con términos nuevos
frase = "Sí señora, el hospital está cerca, será solo un momentito si viene andando"
tokens = tokenizer.tokenize(frase)
ids = tokenizer.convert_tokens_to_ids(tokens)

print("\n🧬 Tokenización de ejemplo:")
for tok, idx in zip(tokens, ids):
    print(f"{tok:<20} → id {idx}")


🧾 Tokens añadidos al vocabulario:
['doña', 'cliente', 'datos', 'fallecimiento', 'gracias', 'hospital', 'momentito', 'propiedad', 'resto', 'doña cliente']

🔢 Tamaño total del vocabulario tras expansión: 50278

🧬 Tokenización de ejemplo:
S                    → id 104
ÃŃ                   → id 1977
Ġse                  → id 842
Ã±                   → id 6303
ora                  → id 4330
,                    → id 6
Ġel                  → id 1615
Ġhospital            → id 1098
Ġest                 → id 3304
Ã¡                   → id 1526
Ġc                   → id 740
erc                  → id 11180
a                    → id 102
,                    → id 6
Ġser                 → id 6821
Ã¡                   → id 1526
Ġsolo                → id 5540
Ġun                  → id 542
Ġ                    → id 1437
momentito            → id 50270
Ġsi                  → id 3391
Ġvi                  → id 12987
ene                  → id 2552
Ġand                 → id 8
ando                 → id 5502


In [17]:
# ------------------------------------
# PASO 5: Guardar dataset salida con nombre estándar
# guardamos el dataset que ya generamos en el paso anterior
# lo arrastramos al siguiente módulo
# ------------------------------------
nombre_salida = os.path.join(
    entorno["ruta_salida"],
    f"dataset_{nombre_modulo.lower()}_{entorno['lote_id']}.csv"
)

utilidades_comunes.guardar_dataset(df_entrada, nombre_salida, logger=logger)

2025-06-28 14:00:46,905 - INFO - 📦 Dataset guardado en: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./salida/dataset_modulo_04_20250614.csv (140 filas, 4 columnas)
INFO:MODULO_04:📦 Dataset guardado en: /content/drive/MyDrive/TFM_EVA_MARTIN/Modulos/MODULO_04/./salida/dataset_modulo_04_20250614.csv (140 filas, 4 columnas)


In [18]:
# ——— Sanity checks finales tras guardar el modelo ———
from transformers import AutoConfig, LongformerTokenizerFast, LongformerForSequenceClassification

#modelo_dir = "…/ruta_salida/modelo_ajustado"

# 1) Comprueba que el num_labels se guardó bien
cfg = AutoConfig.from_pretrained(modelo_dir)
print("✅ num_labels:", cfg.num_labels)  # debe mostrar 3

# 2) Comprueba el tamaño de vocabulario tras resize
tokenizer = LongformerTokenizerFast.from_pretrained(modelo_dir)
print("✅ vocab_size:", len(tokenizer))  # debe reflejar los tokens añadidos

# 3) Comprueba la forma de la capa de clasificación
model = LongformerForSequenceClassification.from_pretrained(modelo_dir, config=cfg)
print("✅ shape out_proj:", model.classifier.out_proj.weight.shape)  # e.g. (3, hidden_size)


✅ num_labels: 3
✅ vocab_size: 50278
✅ shape out_proj: torch.Size([3, 768])
